In [ ]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

# Cake
# 1. The cake is in the kitchen
# 2. The cake is delicious
# 3. The cake is not eaten
# 4. The cake is sweet
# 5. The cake is not sweet
# 6. The cake is not delicious
# 7. The cake is eaten
# 8. The cake is in the stomach

def eat():
    return "eat_cake"

def bake():
    return "bake_cake"

stmap = { 
        Strips(eat(), { "cake": True}, {"cake": False}),
        Strips(bake(), { "cake": False}, {"cake": True})
    }

# all possible actions
feature_domain_dict = { "cake": { True, False} }
strips_domain = STRIPS_domain(feature_domain_dict, stmap)
cake_problem = Planning_problem(strips_domain, { "cake": False}, { "cake": True})
SearcherMPP(Forward_STRIPS(cake_problem)).search()


In [7]:
def searchForSolutions(searcher: SearcherMPP, iterations=1000):
    best_sol = None
    for i in range(iterations):
        print("Iteration: ", i)
        new_sol = searcher.search()
        if new_sol is None:
            break
        if best_sol is None or new_sol.cost < best_sol.cost:
            best_sol = new_sol
    return best_sol

In [10]:
from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

def move(x, y, nx, ny):
    return 'move_from_'+str(x)+str(y)+'_to_'+str(nx)+str(ny)

def attack(x, y):
    return "attack_dragon_at_" + str(x) + "_" + str(y)

def open():
    pass

def collectFire(x, y):
    return "collect_fire_at_" + str(x) + "_" + str(y)

def collectEarth(x, y):
    return "collect_earth_at_" + str(x) + "_" + str(y)

def buildFireball():
    return "build_fireball"

def get_fire(x, y):
    return f"fire_at_{x}_{y}"

def get_earth(x, y):
    return f"earth_at_{x}_{y}"

# nxn grid with player, castle, walls, and dragons
# End goal is kill the dragon and react the castle
# To kill the dragon, player has to be at dragon coordinates and has the fireball
# To build a fireball, the player needs k earth and l fire
# To attack, the player has to be at the same position as a dragon
# Player can open a chest but idk why - skip for now

def initWorld(n, dragon_coords, fire_coords, earth_coords):
    # nxn grid
    feature_domain_dict = { 
        "player": [(i, j) for i in range(n) for j in range(n)], 
        "dragon": (True, False),
        "fireball": (True, False), 
        "collectFire": (True, False),
    }

    for fire in fire_coords:
        feature_domain_dict[get_fire(*fire)] = (True, False)
    
    for earth in earth_coords:
        feature_domain_dict[get_earth(*earth)] = (True, False)

    # all possible player movements
    stmap = {
        Strips(move(x, y, x + dx, y + dy), { "player": (x, y)}, { "player": (x + dx, y + dy)}) 
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]
            for x in range(n) 
            for y in range(n)
            if x + dx in range(n) and y + dy in range(n)
    }

    # collecting fire
    stmap.update({
        Strips(collectFire(*coords), { "player": coords, get_fire(*coords): False, "collectFire": True }, { get_fire(*coords): True, "collectFire": False})
        for coords in fire_coords
    })

    # collecting earth
    stmap.update({
        Strips(collectEarth(*coords), { "player": coords, get_earth(*coords): False, "collectFire": False }, { get_earth(*coords): True, "collectFire": True})
        for coords in earth_coords
    })

    prereq_fireball = {
        "fireball": False,
    }

    for fire in fire_coords:
        prereq_fireball[get_fire(*fire)] = True
    
    for earth in earth_coords:
        prereq_fireball[get_earth(*earth)] = True

    # building a fireball
    stmap.update({
        Strips(buildFireball(), 
        prereq_fireball,
        { "fireball": True })
    })

    # attacking a dragon
    stmap.update({
       Strips(attack(dragon_coords[0], dragon_coords[1]), { "player": dragon_coords, "dragon": True, "fireball": True}, { "dragon": False, "fireball": False }) 
    })

    return STRIPS_domain(feature_domain_dict, stmap)


def searchForSolutions(searcher: SearcherMPP, iterations=1000):
    best_sol = None
    for i in range(iterations):
        print("Iteration: ", i)
        new_sol = searcher.search()
        if new_sol is None:
            break
        if best_sol is None or new_sol.cost < best_sol.cost:
            best_sol = new_sol
    return best_sol

def heur_func(state, goal):
    # calculate the heuristic based on the state
    player = state["player"]
    dragon = state["dragon_coords"]
    # get all the features related to fire locations
    firekeys = list(filter(lambda x: not state[x], filter(lambda x: "fire_at" in x, state.keys())))
    earthkeys = list(filter(lambda x: not state[x], filter(lambda x: "earth_at" in x, state.keys())))
    
    collectFire = state["collectFire"]
    fireball = state["fireball"]
    castle = goal["player"]

    def castle_dist():
        return abs(player[0] - castle[0]) + abs(player[1] - castle[1])
    
    def dragon_dist():
        return abs(player[0] - dragon[0]) + abs(player[1] - dragon[1])

    def fire_dist():
        firelocations = [f.split("_")[2:] for f in firekeys]
        return min([abs(player[0] - int(f[0])) + abs(player[1] - int(f[1])) for f in firelocations])
    
    def earth_dist():
        earthlocations = [f.split("_")[2:] for f in earthkeys]
        return min([abs(player[0] - int(f[0])) + abs(player[1] - int(f[1])) for f in earthlocations])

    # distance = manhattan distance
    # if dragon dead, heuristic is the distance to the castle
    if not dragon:
        return castle_dist()
    # if dragon alive, and player has fireball, heuristic is distance to the dragon + distance to the castle
    elif fireball or (len(firekeys) + len(earthkeys) == 0):
        return dragon_dist()
    # if dragon alive, player doesn't have fireball, heuristic is distance to the dragon + distance to the nearest fire/earth (whatever needs to be collected)
    elif collectFire:
        return fire_dist()
    else:
        return earth_dist()


def problemCreator(dragon, fire, earth, player_coords, castle, iters=100):
    world1 = initWorld(10, dragon, fire, earth)
    start_state = {
        "player": player_coords,
        "dragon": True,
        "fireball": False,
        "collectFire": True,
        "dragon_coords": dragon,
    }

    for f in fire:
        start_state[get_fire(*f)] = False

    for e in earth:
        start_state[get_earth(*e)] = False

    goal_state = {
        "player": castle,
        "dragon": False
    }

    problem = Planning_problem(
        world1, 
        start_state, 
        goal_state
    )
    sol = searchForSolutions(SearcherMPP(Forward_STRIPS(problem, heur=heur_func)), iters)
    print(sol)
    print(f"Cost of the solution: {sol.cost}")

In [11]:
dragon = (1, 3)
fire = [(2, 2)]
earth = [(7, 5)]
player_coords = (1, 1)
castle = (4, 4)
problemCreator(dragon, fire, earth, player_coords, castle, 10)

Iteration:  0
Solution: {'player': (1, 1), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_2': False, 'earth_at_7_5': False}
   --move_from_11_to_21--> {'player': (2, 1), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_2': False, 'earth_at_7_5': False}
   --move_from_21_to_22--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_2': False, 'earth_at_7_5': False}
   --collect_fire_at_2_2--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (1, 3), 'fire_at_2_2': True, 'earth_at_7_5': False}
   --move_from_22_to_32--> {'player': (3, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (1, 3), 'fire_at_2_2': True, 'earth_at_7_5': False}
   --move_from_32_to_33--> {'player': (3, 3), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (1, 3), 'fire_at_2_2':

In [23]:
dragon = (1, 3)
fire = [(2, 4), (8, 8)]
earth = [(7, 5)]
player_coords = (1, 1)
castle = (4, 4)
problemCreator(dragon, fire, earth, player_coords, castle, 10)

Iteration:  0
Solution: {'player': (1, 1), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_11_to_21--> {'player': (2, 1), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_21_to_22--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_22_to_23--> {'player': (2, 3), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --move_from_23_to_24--> {'player': (2, 4), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (1, 3), 'fire_at_2_4': False, 'fire_at_8_8': False, 'earth_at_7_5': False}
   --collect_fire_at_2_4--> {'pla

In [22]:
dragon = (0, 0)
fire = [(1, 1), (1, 2)]
earth = [(2, 1), (9, 9)]
player_coords = (2, 2)
castle = (9, 8)
problemCreator(dragon, fire, earth, player_coords, castle, 10)

Iteration:  0
Solution: {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': False, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --move_from_22_to_12--> {'player': (1, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': False, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --collect_fire_at_1_2--> {'player': (1, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': True, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --move_from_12_to_22--> {'player': (2, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (0, 0), 'fire_at_1_1': False, 'fire_at_1_2': True, 'earth_at_2_1': False, 'earth_at_9_9': False}
   --move_from_22_to_32--> {'player': (3, 2), 'dragon': True, 'fireball': False, 'collectFire': False, 'dragon_coords': (0, 0), 'fire_

## Hard 

In [33]:
dragon = (2, 11)
fire = [(2, 5), (5, 1), (10, 8), (1, 5)]
earth = [(4, 2), (0, 6), (3, 11)]
player_coords = (7, 2)
castle = (8, 8)
problemCreator(dragon, fire, earth, player_coords, castle, 12)

Iteration:  0
Solution: {'player': (5, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 2), 'fire_at_2_5': False, 'earth_at_4_2': False}
   --move_from_52_to_42--> {'player': (4, 2), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 2), 'fire_at_2_5': False, 'earth_at_4_2': False}
   --move_from_42_to_43--> {'player': (4, 3), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 2), 'fire_at_2_5': False, 'earth_at_4_2': False}
   --move_from_43_to_44--> {'player': (4, 4), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 2), 'fire_at_2_5': False, 'earth_at_4_2': False}
   --move_from_44_to_34--> {'player': (3, 4), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 2), 'fire_at_2_5': False, 'earth_at_4_2': False}
   --move_from_34_to_35--> {'player': (3, 5), 'dragon': True, 'fireball': False, 'collectFire': True, 'dragon_coords': (2, 2), 'fire_at_2_5': F

In [25]:
dragon = (10, 11)
fire = [(2, 2), (10, 1), (1, 11), (1, 9), (0, 0)]
earth = [(7, 5), (6, 6), (9, 11), (7, 9)]
player_coords = (11, 10)
castle = (5, 5)
problemCreator(dragon, fire, earth, player_coords, castle, 12)

Iteration:  0
No (more) solutions. Total of 1 paths expanded.
None


AttributeError: 'NoneType' object has no attribute 'cost'

In [27]:
dragon = (10, 11)
fire = [(2, 2), (6, 1), (5, 9), (1, 9), (2, 8)]
earth = [(7, 5), (4, 6), (3, 11), (7, 9), (11, 11)]
player_coords = (11, 10)
castle = (5, 5)
problemCreator(dragon, fire, earth, player_coords, castle, 12)

Iteration:  0
No (more) solutions. Total of 1 paths expanded.
None


AttributeError: 'NoneType' object has no attribute 'cost'

In [ ]:
#    --move_from_13_to_23--> {'player': (2, 3), 'dragon': False, 'fireball': False, 'collectFire': True, 'fire_at_2_2': True, 'fire_at_1_1': True, 'earth_at_7_5': True, 'earth_at_6_6': True}
#    --move_from_23_to_33--> {'player': (3, 3), 'dragon': False, 'fireball': False, 'collectFire': True, 'fire_at_2_2': True, 'fire_at_1_1': True, 'earth_at_7_5': True, 'earth_at_6_6': True}
#    --move_from_33_to_43--> {'player': (4, 3), 'dragon': False, 'fireball': False, 'collectFire': True, 'fire_at_2_2': True, 'fire_at_1_1': True, 'earth_at_7_5': True, 'earth_at_6_6': True}
#    --move_from_43_to_44--> {'player': (4, 4), 'dragon': False, 'fireball': False, 'collectFire': True, 'fire_at_2_2': True, 'fire_at_1_1': True, 'earth_at_7_5': True, 'earth_at_6_6': True}

In [ ]:
# ; Shakey's world, or versions thereof, is an AI classic. A robot, named Shakey, is moving around in a set of rooms, 
# ; connected by doors. 
# ; In the rooms there are light switches, so that the light in the room can be on or off. Spread throughout this world 
# ; there are a number of big boxes and a number of smaller balls. 
# ; Also there are specific locations in each room where the boxes and robots can be present initially(init1,init2,init3) and where they
# ; need to be transfered finally(fin1,fin2,fin3).
# ; Here's an example of a rather small world layout: 
# ;
# ; The follwing restrictions apply to Shakey's actions: 
# ;
# ; Shakey can carry small balls, but only one at a time because he has only one gripper. 
# ; For Shakey to be able to find a door and  move to another room, the light must be on in that room. 
# ; Shakey can not carry boxes, but can push them around. 
# ; To switch the light in a room on or off Shakey must putdown the ball if it was carrying otherwise he can not 
# ; turn on the light switch. 

from stripsProblem import Planning_problem, STRIPS_domain, Strips
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS

def pickup(id, room):
    return f"pickup_ball_{id}_{room}"

def putdown(id, room):
    return f"putdown_ball_{id}_{room}"

def pushbox(x1, y1, x2, y2):
    return f"pushbox_from_{x1}_{y1}_to_{x2}_{y2}" 

def move_bet_rooms(r1, r2):
    return f"tp_from_{r1}_to_{r2}"

def move_in_room(x1, y1, x2, y2):
    return f"move_from_{x1}_{y1}_to_{x2}_{y2}"

def turn_on_light(r1):
    return f"turn_on_in_room_{r1}"

def get_light(room):
    return f"light_{room}"

def get_ball(id, room):
    return f"ball_{id}_{room}"

def is_ball_picked(id, room):
    return get_ball(id, room) + "_picked"

def initProblemDefinition(
        size, 
        init_coords, 
        light_coords, 
        rooms, 
        final_coords,
        balls,
        boxes
    ):

    if (init_coords[0] not in range(size) or
        init_coords[1] not in range(size)):
        raise ValueError("Invalid initial position")

    if (light_coords[0] not in range(size) or
        light_coords[1] not in range(size)):
        raise ValueError("Invalid light coordinates")
    
    if (final_coords[0] not in range(size) or
        final_coords[1] not in range(size)):
        raise ValueError("Invalid final position")
    
    if (len(rooms) < 2):
        raise ValueError("At least two rooms are required")
    
    if (len(balls) < 1):
        raise ValueError("At least one ball is required")
    
    if (len(boxes) < 1):
        raise ValueError("At least one box is required")
    
    for ball in balls:
        if (ball[0] not in range(size) or
            ball[1] not in range(size)):
            raise ValueError("Invalid ball coordinates")
    
    for box in boxes:
        if (box[0] not in range(size) or
            box[1] not in range(size)):
            raise ValueError("Invalid box coordinates")
    
    # final coords is the postiion of the trash where robot has to move the balls and boxes
    
    # rooms are placed in a line
    # there is a bidirectional connection between two adjacent rooms
    # room0 <-> room1 <-> ... <-> room(n - 1) <-> room(n)
    adj = {
        rooms[i]: [rooms[i + j] for j in [-1, 1] if i + j in range(len(rooms))] for i in range(len(rooms))
    }

    feature_domain_dict = {
        "room": rooms,
        "robot": [(x, y) for x in range(size) for y in range(size)],
    }

    # lights
    for room in rooms:
        feature_domain_dict[get_light(room)] = (True, False)

    # balls
    for i in range(len(balls)):
        for room in rooms:
            feature_domain_dict[get_ball(i, room)] = [balls[i], final_coords]
            feature_domain_dict[is_ball_picked(i, room)] = (True, False)

    # boxes
    # for box in boxes:
    #     for room in rooms:
    #         feature_domain_dict[f"box_at_{box[0]}_{box[1]}_{room}"] = (True, False)

    # defining robot movements within the room
    stmap = {
        Strips(move_in_room(x, y, x + dx, y + dy), { "robot": (x, y)}, { "robot": (x + dx, y + dy)}) 
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]
            for x in range(size) 
            for y in range(size)
            if x + dx in range(size) and y + dy in range(size)
    }

    # turning on light
    # the robot has to be at lights coords to turn them on
    stmap.update({
        Strips(turn_on_light(room), { "room": room, get_light(room): False, "robot": light_coords }, { get_light(room): True })
        for room in rooms
    })

    # move between rooms definied in adj list
    # it is allowed only if the light is turned on
    # starting position in the new room is `init_coords`
    stmap.update({  
        Strips(move_bet_rooms(fr, to), { "room": fr, get_light(fr): True }, { "room": to, "robot": init_coords }) 
        for fr, adj_list in adj.items() for to in adj_list
    })

    # pickup a ball from its original position
    stmap.update({
        Strips(
            pickup(i, room), 
        { 
            "robot": coords,
            is_ball_picked(i, room): False,
            get_ball(i, room): coords, 
            "room": room 
        }, { is_ball_picked(i, room): True })
        for i, coords in enumerate(balls)
        for room in rooms
    })

    # # putdown ball in the trash at final_coords
    stmap.update({
        Strips(
            putdown(i, room), 
        { 
            "robot": final_coords,
            is_ball_picked(i, room): True, 
            "room": room 
        }, 
        { is_ball_picked(i, room): False, get_ball(i, room): final_coords }
        )
        for i, _ in enumerate(balls)
        for room in rooms
    })

    return STRIPS_domain(feature_domain_dict, stmap)


size = 5
light_coords = (0, 0)
init_coords = (2, 2)
n_rooms = 2
rooms = [f"room{i}" for i in range(n_rooms)]
final_coords = (4, 4)
balls = [(1, 1)]
boxes = [(0, 0), (4, 4)]

problem_definition = initProblemDefinition(size, light_coords, init_coords, rooms, final_coords, balls, boxes)

start_state = {
    "robot": init_coords,
    "room": rooms[0]
}

for room in rooms:
    start_state[get_light(room)] = False

for i in range(len(balls)):
    for room in rooms:
        start_state[is_ball_picked(i, room)] = False
        start_state[get_ball(i, room)] = balls[i]

goal_state = {}

for room in rooms:
    goal_state[f"light_{room}"] = True

# for i in range(len(balls)):
#     for room in rooms:
#         goal_state[get_ball(i, room)] = final_coords

problem = Planning_problem(
    problem_definition, 
    start_state, 
    goal_state
)

sol = SearcherMPP(Forward_STRIPS(problem)).search()
print(sol)